In [1]:
import cv2
import numpy as np

# 모폴로지 연산

## 침식과 팽창

In [9]:
src = cv2.imread('./data/milkdrop.bmp', cv2.IMREAD_GRAYSCALE)
thresh, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

erode = cv2.erode(src_bin, None)
dilate = cv2.dilate(src_bin, None)

cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('erode', erode)
cv2.imshow('dilate', dilate)
cv2.waitKey()
cv2.destroyAllWindows()

## 열기와 닫기

In [13]:
src = cv2.imread('./data/milkdrop.bmp', cv2.IMREAD_GRAYSCALE)
thresh, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 열기 : 침식 -> 팽창
opening = cv2.morphologyEx(src_bin, cv2.MORPH_OPEN, None, iterations = 1)

# 닫기 : 팽창 -> 침식
closing = cv2.morphologyEx(src_bin, cv2.MORPH_CLOSE, None, iterations = 1)


#cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('opening', opening)
cv2.imshow('closing', closing)

cv2.waitKey()
cv2.destroyAllWindows()

# 레이블링과 외곽선 검출

## 레이블링 기본

In [14]:
src = np.array([[0, 0, 1, 1, 0, 0, 0, 0],
                [1, 1, 1, 1, 0, 0, 1, 0],
                [1, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 1, 0],
                [0, 0, 0, 1, 1, 1, 1, 0],
                [0, 0, 0, 1, 0, 0, 1, 0],
                [0, 0, 1, 1, 1, 1, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0]]).astype(np.uint8)

src = src * 255
cnt, labels = cv2.connectedComponents(src)

print(cnt)
print(labels)

4
[[0 0 1 1 0 0 0 0]
 [1 1 1 1 0 0 2 0]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 0 3 3 0]
 [0 0 0 3 3 3 3 0]
 [0 0 0 3 0 0 3 0]
 [0 0 3 3 3 3 3 0]
 [0 0 0 0 0 0 0 0]]


In [24]:
src = cv2.imread('./data/circles.jpg', cv2.IMREAD_GRAYSCALE)
thresh, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

dst[labels == 0] = (0, 255, 255)
dst[labels == 1] = (0, 0, 255)
dst[labels == 2] = (255, 0, 255)
dst[labels == 3] = (0, 255, 0)

In [25]:
stats

array([[     0,      0,    512,    512, 222722],
       [   308,     86,    125,    125,  12281],
       [   153,    145,    152,    152,  18150],
       [   292,    338,    107,    107,   8991]], dtype=int32)

In [22]:
cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 레이블링 응용

In [30]:
src = cv2.imread('./data/circles.jpg', cv2.IMREAD_GRAYSCALE)
thresh, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR) # 객체별로 색깔을 다르게 표시할 3차원 도화지 준비

for i in range(1, cnt): # 객체 1, 2, 3
    b = np.random.randint(0, 256)
    g = np.random.randint(0, 256)
    r = np.random.randint(0, 256)
    dst[labels == i] = (b, g, r)
    
    # bounding box --> rectangle
    x, y, width, height, area = stats[i]
    cv2.rectangle(dst, (x, y), (x+width, y+height), (0, 0, 255))
    
    # centroids --> circle
    cx, cy = centroids[i]
    cv2.circle(dst, (int(cx), int(cy)), 5, (255, 0, 0), -1)

cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [41]:
src = cv2.imread('./data/keyboard.bmp')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

thresh, src_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = src.copy()

for i in range(1, cnt): # 0번 배경 제외, 1~37번 객체까지 반복
    x, y, width, height, area = stats[i]
    if area > 20:
        cv2.rectangle(dst, (x, y), (x + width, y + height), (0, 255, 255))



In [42]:
cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 외곽선 검출과 그리기

In [70]:
src = cv2.imread('./data/contours.bmp')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
thresh, gray_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

dst = src.copy()
contours, hierachy = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# cv2.drawContours(dst, contours, 0, (255, 0, 0), 2)
# cv2.drawContours(dst, contours, 1, (255, 0, 0), 2)
# cv2.drawContours(dst, contours, 2, (255, 255, 0), 2)
# cv2.drawContours(dst, contours, 3, (255, 255, 0), 2)
# cv2.drawContours(dst, contours, 4, (255, 255, 0), 2)

for i in range(len(contours)):
    b = np.random.randint(0, 256)
    g = np.random.randint(0, 256)
    r = np.random.randint(0, 256)
    cv2.drawContours(dst, contours, i, (b, g, r), 2)


cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [100]:
src = cv2.imread('./data/thumbs_up_down.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

thresh, gray_bin = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)

dst = src.copy()
contours, hierachy = cv2.findContours(gray_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(dst, contours, -1, (255, 255, 0), 3)

# for i in range(len(contours)):
#     b = np.random.randint(0, 256)
#     g = np.random.randint(0, 256)
#     r = np.random.randint(0, 256)
#     cv2.drawContours(dst, contours, i, (b, g, r), 2)

cv2.imshow('src', src)
cv2.imshow('gray', gray)
cv2.imshow('gray_bin', gray_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


In [89]:
contours

(array([[[0, 0]],
 
        [[0, 1]],
 
        [[0, 2]],
 
        ...,
 
        [[3, 0]],
 
        [[2, 0]],
 
        [[1, 0]]], dtype=int32),)

In [106]:
left_hand = contours[1]
x, y, w, h = cv2.boundingRect(left_hand)

dst2 = src.copy()

cv2.rectangle(dst2, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()


# 객체 검출

## 캐스케이스 분류기와 얼굴 검출

**opencv**
https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html

**haar cascade classifier**
https://towardsdatascience.com/viola-jones-algorithm-and-haar-cascade-classifier-ee3bfb19f7d8

https://webnautes.tistory.com/1352 (한글 블로그)

**adaboost**
https://www.youtube.com/watch?v=LsK-xG1cLYA (유튜브)

In [ ]:
# detectMultiScale
# scaleFactor : 검색 윈도우 확대 비율 (default =1.1)
# minNeighbors : 검출 영역으로 선택하기 위한 최소 검출 횟수 (default = 3)
# minSize : 검출할 객체의 최소크기
# maxSize : 검출할 객체의 최대크기

## HOG 알고리즘과 보행자 검출

**HOG Descriptor**
https://docs.opencv.org/4.x/d5/d33/structcv_1_1HOGDescriptor.html#a723b95b709cfd3f95cf9e616de988fc8